### moduls install

In [103]:
!pip install tensorflow==2.9.0
!pip install "numpy<2.0"
!pip install matplotlib seaborn scikit-learn opencv-python-headless

2660.33s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2666.20s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2672.00s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


### import moduls

In [104]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

### ⭐️ dir setting

In [105]:
model_dir = '/home/nanopore/바탕화면/hajung/watermelon-disease/results/efficient_watermelonAug_250408' # ⭐️ 여기 경로 확인
structure_path = os.path.join(model_dir, 'model_structure.json')
weights_path = os.path.join(model_dir, 'model_weights.h5')
config_path = os.path.join(model_dir, 'config.json')
class_info_path = os.path.join(model_dir, 'class_info.json')

### model loading

In [106]:
# class info
def load_class_info(class_info_path):
    # class_info.json 로드
    with open(os.path.join(model_dir, 'class_info.json')) as f:
        class_info = json.load(f)  

    # 클래스 인덱스 → 라벨 딕셔너리 생성
    idx_to_class = {entry['index']: entry['class'] for entry in class_info}
    
    # 입력 이미지 크기
    img_size = (class_info[0]['height'], class_info[0]['width'])
    
    return idx_to_class, img_size

In [117]:

# 모델 + 클래스 정보 불러오기 (추론 전용)
def load_inference_model(model_dir):
    # 모델 구조 + 가중치
    with open(os.path.join(model_dir, 'model_structure.json')) as f:
        model = model_from_json(f.read())
    model.load_weights(os.path.join(model_dir, 'model_weights.h5'))

    return model


### input data

In [118]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import numpy as np

In [119]:

# 전처리 함수
def preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=img_size)  # 모델 입력 사이즈에 맞게 수정!
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # 정규화
    img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가
    
    return img_array

### run

In [120]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [121]:
# path setting
structure_path = 'best_model/model_structure.json'
weights_path = 'best_model/model_weights.h5'
config_path = 'best_model/config.json'
class_names_path = 'best_model/class_names.json'


In [122]:
# 후처리 함수
def decode_prediction(preds, idx_to_class):
    pred_index = np.argmax(preds, axis=1)[0]
    pred_class = idx_to_class[pred_index]
    confidence = float(preds[0][pred_index])
    return pred_class, confidence

# 이미지 확인
def image_print(img):
    plt.imshow(img[0])
    plt.axis('off')
    plt.show()
    return img

# 예측 함수
def predict_image(model, img, idx_to_class):
    preds = model(img)
    pred_class, confidence = decode_prediction(preds, idx_to_class)
    return pred_class, confidence

#### ⭐️ model activate

In [123]:
print(type(model))

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [124]:
# call variations
idx_to_class, img_size = load_class_info('경로/파일이름')
img_path = '/home/nanopore/바탕화면/hajung/input/img_MKH180810001_1.jpg'  # ⭐️ 예측할 이미지 경로
img = preprocess_image(img_path, img_size)
model = load_inference_model(model_dir)

# run model
pred_class, confidence = predict_image(model, img, idx_to_class)

print(f"📌 예측 클래스: {pred_class} ({confidence * 100:.2f}%)")

📌 예측 클래스: Mosaic_Virus (99.68%)
